# 07wk-1: (합성곱신경망) – CNN 자랑, CNN 핵심레이어

최규빈  
2025-04-16

<a href="https://colab.research.google.com/github/guebin/DL2025/blob/main/posts/06wk-2.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" style="text-align: left"></a>

# 1. 강의영상

In [ ]:
# {{<video https://youtu.be/playlist?list=PLQqh36zP38-wcPiCEdYML9-6-Xv5RVbso&si=BbNo6mwCHqwOV0FS>}}

# 2. Imports

In [2]:
import torch
import torchvision
import matplotlib.pyplot as plt

In [3]:
plt.rcParams['figure.figsize'] = (4.5, 3.0)

# 3. CNN 자랑

## A. 성능좋음

*Fashion MNIST*

In [47]:
train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True)
train_dataset = torch.utils.data.Subset(train_dataset, range(5000))
test_dataset = torch.utils.data.Subset(test_dataset, range(1000))
to_tensor = torchvision.transforms.ToTensor()
X = torch.stack([to_tensor(img) for img, lbl in train_dataset]).to("cuda:0")
y = torch.tensor([lbl for img, lbl in train_dataset])
y = torch.nn.functional.one_hot(y).float().to("cuda:0")
XX = torch.stack([to_tensor(img) for img, lbl in test_dataset]).to("cuda:0")
yy = torch.tensor([lbl for img, lbl in test_dataset])
yy = torch.nn.functional.one_hot(yy).float().to("cuda:0")

*발악수준으로 설계한 신경망*

In [56]:
net = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(784,2048),
    torch.nn.ReLU(),
    torch.nn.Linear(2048,10)
).to("cuda")
loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(net.parameters())

In [49]:
for epoc in range(1,500):
    #1
    logits = net(X)
    #2
    loss = loss_fn(logits, y) 
    #3
    loss.backward()
    #4 
    optimizr.step()
    optimizr.zero_grad()

In [50]:
(net(X).argmax(axis=1) == y.argmax(axis=1)).float().mean()

In [51]:
(net(XX).argmax(axis=1) == yy.argmax(axis=1)).float().mean()

*대충대충 설계한 합성곱신경망*

In [52]:
net = torch.nn.Sequential(
    torch.nn.Conv2d(1,16,2),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(2),
    torch.nn.Flatten(),
    torch.nn.Linear(2704,10),
).to("cuda")
loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(net.parameters())

In [53]:
for epoc in range(1,500):
    #1
    logits = net(X)
    #2
    loss = loss_fn(logits, y) 
    #3
    loss.backward()
    #4 
    optimizr.step()
    optimizr.zero_grad()

In [54]:
(net(X).argmax(axis=1) == y.argmax(axis=1)).float().mean()

In [55]:
(net(XX).argmax(axis=1) == yy.argmax(axis=1)).float().mean()

## B. 파라메터적음

In [76]:
net1 = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(784,2048),
    torch.nn.ReLU(),
    torch.nn.Linear(2048,10)
)
net2 = torch.nn.Sequential(
    torch.nn.Conv2d(1,16,2),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(2),
    torch.nn.Flatten(),
    torch.nn.Linear(2704,10),
)

In [77]:
net1_params = list(net1.parameters())
net2_params = list(net2.parameters())

In [78]:
for params in net1_params:
    print(params.shape)

torch.Size([2048, 784])
torch.Size([2048])
torch.Size([10, 2048])
torch.Size([10])

In [79]:
2048*784 + 2048 + 10*2048 + 10 

In [80]:
for params in net2_params:
    print(params.shape)

torch.Size([16, 1, 2, 2])
torch.Size([16])
torch.Size([10, 2704])
torch.Size([10])

In [81]:
16*1*2*2 + 16 + 10*2704 + 10

## C. 유명함

`-` <https://brunch.co.kr/@hvnpoet/109>

# 4. CNN 핵심레이어

## A. `torch.nn.ReLU`

**(예시1) 연산방법**

In [126]:
img = torch.randn(1,1,5,5) # 흑백이미지한장
relu = torch.nn.ReLU()

In [127]:
img # - 값도 있지만 이미지라고 생각하자..

In [128]:
relu(img)

## B. `torch.nn.MaxPool2d`

**(예시1) 연산방법, kernel_size 의 의미**

In [172]:
img = torch.rand(1,1,4,4) # 흑백이미지한장
mp = torch.nn.MaxPool2d(kernel_size=2)

In [173]:
img

In [174]:
mp(img)

**(예시2) 이미지크기와 딱 맞지않는 커널일경우?**

In [71]:
img = torch.rand(1,1,5,5) # 흑백이미지한장
mp = torch.nn.MaxPool2d(kernel_size=2)

In [72]:
img

In [73]:
mp(img)

**(예시3) 정사각형이 아닌 커널**

In [61]:
img = torch.rand(1,1,4,4) # 흑백이미지한장
mp = torch.nn.MaxPool2d(kernel_size=(2,4))

In [62]:
img

In [63]:
mp(img)

## C. `torch.nn.Conv2d`

**(예시1) 연산방법, stride=2**

In [89]:
img = torch.rand(1,1,4,4) # 흑백이미지한장
conv = torch.nn.Conv2d(in_channels=1,out_channels=1,kernel_size=2,stride=2)

In [90]:
img

In [91]:
conv(img)

??

In [114]:
conv.weight, conv.bias

In [168]:
(img[:,  :,  :2,  :2] * conv.weight.data).sum() + conv.bias, conv(img)

In [169]:
(img[:,  :,  :2,  2:] * conv.weight.data).sum() + conv.bias, conv(img)

In [170]:
(img[:,  :,  2:,  :2] * conv.weight.data).sum() + conv.bias, conv(img)

In [171]:
(img[:,  :,  2:,  2:] * conv.weight.data).sum() + conv.bias, conv(img)

**(예시2) 이동평균**

In [228]:
img = torch.arange(1,17).float().reshape(1,1,4,4)
conv = torch.nn.Conv2d(in_channels=1,out_channels=1,kernel_size=2,stride=1,bias=False)
conv.weight.data = conv.weight.data*0+1/4

In [230]:
img

In [231]:
conv(img)

**(예시3) 2개의 이미지**

In [232]:
imgs = torch.arange(1,33).float().reshape(2,1,4,4)
conv = torch.nn.Conv2d(in_channels=1,out_channels=1,kernel_size=2,stride=1,bias=False)
conv.weight.data = conv.weight.data*0+1/4

In [234]:
imgs

In [235]:
conv(imgs)

**(예시4) 2개의 out_channels**

In [257]:
img = torch.arange(1,33).float().reshape(2,1,4,4)
conv = torch.nn.Conv2d(in_channels=1,out_channels=2,kernel_size=2,stride=1,bias=False)
conv.weight.data[0] = conv.weight.data[0]*0 + 1/4
conv.weight.data[1] = conv.weight.data[1]*0 

In [258]:
img

In [259]:
conv(img)